# dynamo db - create table

In [1]:
import os
import json
import time
import boto3
import tweepy
import pprint
import datetime
import numpy as np
import pandas as pd
import botocore.session
from boto3.dynamodb.conditions import Key, Attr

with open("../creds.json", "r") as f:
    creds = json.load(f)
    f.close()
    
print("imported modules.")

ModuleNotFoundError: No module named 'boto3'

## dynamodb client

In [4]:
client = boto3.client('dynamodb',
  endpoint_url = "http://xanaxprincess.asuscomm.com:8001",
  aws_access_key_id=creds["aws-access-key"],
  aws_secret_access_key=creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://xanaxprincess.asuscomm.com:8001", \
            region_name='us-east-2')

print("created dynamo client.")

created dynamo client.


## create dynamo table

In [15]:
table = dynamodb.create_table(
    TableName='tweets',
    KeySchema=[
        {
            'AttributeName': 'user',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'created_at',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'user',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'created_at',
            'AttributeType': 'S'
        }

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
)

print("Table status:", table.table_status)

Table status: ACTIVE


## wait for table creation

In [18]:
session = botocore.session.get_session()
dynamodb_session = session.create_client('dynamodb', \
                    region_name='us-east-2',
                    endpoint_url = "http://xanaxprincess.asuscomm.com:8001",
                    aws_access_key_id=creds["aws-access-key"],
                    aws_secret_access_key=creds["aws-secret-access-key"]) # low-level client

waiter = dynamodb_session.get_waiter('table_exists')
waiter.wait(TableName="tweets")

## scan tweets table

In [13]:
tweets = dynamodb.Table("tweets")

response = tweets.scan()

In [15]:
response

{'Items': [{'favorites': Decimal('3'),
   'created_at': '2023-01-24T18:09:56Z',
   'tweet': '16 years \U0001fae0 https://t.co/pmXRoXOdWz',
   'id': Decimal('1617947826671190017'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus'},
  {'favorites': Decimal('1'),
   'created_at': '2023-01-24T18:23:49Z',
   'tweet': '16 years, noooooooo https://t.co/8poAs8c6dr',
   'id': Decimal('1617951319050039299'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus'},
  {'favorites': Decimal('4'),
   'created_at': '2023-01-24T19:51:02Z',
   'tweet': 'gosh this stupid hellsite (affectionate) is my favorite bar and all and i love doing shitpost jello shots with the… https://t.co/2PP4R0jnJC',
   'id': Decimal('1617973267859673088'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus'},
  {'favorites': Decimal('0'),
   'created_at': '2023-01-24T23:56:36Z',
   'tweet': 'RT @non_bunnary: @FizzyDog_ The address for her website is .gay\n\nIt’s very important and I think we shouldn’t forget it',


In [16]:
! aws dynamodb list-tables --endpoint-url http://xanaxprincess.asuscomm.com:8001

{
    "TableNames": [
        "tweets"
    ]
}


In [18]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://xanaxprincess.asuscomm.com:8001

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "user",
                "AttributeType": "S"
            },
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1674535588.165,
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": 0.0,
            "LastDecreaseDateTime": 0.0,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 105779,
        "ItemCount": 508,
        "TableArn": "arn:aws:dynamodb:ddblocal:000000000000: